# Week 6 lecture notes (2/13/25) Unit Testing + Threads & Pipelines
- the dsh_cli needs no code changes
- all code implementation is in the lib file.


### unit tests
#! = is a "shabang-line comment" with a path to some executable which defines were the shell should be run
- good test naming should describe the test and what it is looking for.
    ex:
    @test "pwd should return 0" {
        run pwd
        ["$status" -eq 0] # anything in bats surounded by [] is an assertion and must resolve to true or will error.
    }
    - status is a variable that is populated by bats tests that contain the output of your "run" command
    - testing that status is equal to zero
    - all assertions in a test must be true for the test to pass.
- to print a variable or string in a test use the "echo" method.
- in vats $output should contain the output of your script via the call made by "run some-command"
- when creating unit tests for asserting the string output of a command it is best practice to strip the whitespace from the output and use that for the assertion just to allow for easier matching and preventing tests from failing with whitespace.
    - ex. stripped_output = $(echo "$output" | tr -d '[:space:]')
- commands that read from stdin can be provided multi-line input using a "heredoc" whihc is the same way in which yoy can test a "run" of 'dsh'
    ex. run tee <<FOO
        hello,
        world
        FOO # runs this multi-line command => run till you hit "FOO"
    ex2. run ./dsh <<EOF 
    - common unit testing use of a heredoc where it is run until hitting EOF
- You should create tests first with criteria for passing, see them fail and then write your code until they pass

### Threads
- a thread => a lightweight thread that shares proccess address space but has an isolated stack
    - benefits: lightweight, process isolation, shared address space
    - shared virtual memmory (heap,data,text), stack is not shared (holds execution context.(task struct))
- synchronization is the practice of managing access to resources amongst multiple threads
    - the MIGHTY Mutex => ensures mutually exclusive access - only one thread at a time. Write and read saftey implemented like a "lock" on shared memory(heap, globals, static)
        - when a thread successfully "locks" a mutex or gets access, it "owns" the mutex until it unlocks it so other threads can access
        - technically a mutex is a kind of semaphore
    - The SEMAPHORE=> a gatekeeper that only allows N number of threads to perform some operation at the same time - allows max amount of threads to perform a certain action
        - an example of this is a connection pool which is max amount of concurrent connections to a database.
        - acts as a counting mechanism to manage numbner of threads processing concurrently - doesn't prevent race conditions within the critical section unless additional synchronization is used (a mutex or lock)
- when working with multiple threads the "pthread_join(thread, NULL)" will stop the main process and wait for the child proccess to finish -> then continue with the main thread
    - if you spawn multiple children you must loop through them checking of if the are complete with a "join()"

